# HomeWork

In [ ]:
import tensorflow as tf
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
import numpy as np
from os import listdir,path
import random
import shutil

# Variables Declare

In [ ]:
img_size=(128,128)
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
trainDataPath = '/home/train'
testDataPath = '/home/test'
validationDataPath = '/home/validation'
batchSize = 32
classMode = 'categorical'
classficationLabels = listdir(trainDataPath)
arrX = []
arrY = []
datasetFolderName = path.join('')
trainData = None
testData = None
validData = None

# Load the data

In [ ]:
def loadData(type):
    if type == 'train':
        
        trainData = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            zoom_range=0.20,
            fill_mode="nearest"
        ).flow_from_directory(
            trainDataPath,
            target_size=img_size,
            batch_size=batchSize,
            class_mode= classMode
        )
        return trainData
    elif type == 'test':
        testData = generator.flow_from_directory(
            testDataPath,
            target_size=img_size,
            batch_size=batchSize,
            class_mode= classMode
        )
        return testData
    elif type == 'valid' :
        validData = generator.flow_from_directory(
            validationDataPath,
            target_size=img_size,
            batch_size=batchSize,
            class_mode= None,
            shuffle=False
        )
        return validData



# Define the Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)), # Layer 1
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # Layer 2
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'), # Layer 3
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(), # Full c. layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Kfold Functions Define

In [ ]:
def move(src,des,spiltRate):
    srcFiles = listdir(src)
    if(len(srcFiles) != 0):
        numberOfTransfer = int(len(srcFiles)*spiltRate)
        transferIndex = random.sample(range(0,len(srcFiles)),numberOfTransfer)
        for index in transferIndex :
            shutil.move(path.join(src,str(srcFiles[index])),
                        path.join(des,str(srcFiles[index])))
    else:
        print("No file move")

def moveClass(src=trainDataPath,des=validationDataPath,spiltRate=0.2):
    for label in classficationLabels:
        move(path.join(src,label),
             path.join(des,label),
             spiltRate
            )
        
def fillArr():
    X=[]
    Y=[]
    for folder in classficationLabels:
        srcFiles = listdir(path.join(trainDataPath,folder))
        yLabel = 0
        for i in range(len(classficationLabels)):
            if(folder == classficationLabels[i]):
                yLabel = i
        for imageName in srcFiles:
            X.append(imageName)
            Y.append(yLabel)
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X,Y

def meansure(anwser,predict):
    accuracy = accuracy_score(anwser,predict)
    precision = precision_score(anwser,predict,average='weighted')
    f1 = f1_score(anwser,predict,average='weighted')
    print("Accuracy : %d\nPrecision : %d\nf1Score : %d\n" %(accuracy,precision,f1))
    confusionMatrix = confusion_matrix(anwser,predict)
    print("Confusion_Matrix : \n{}".format(confusionMatrix))
    return accuracy,precision,f1


# Kfold

In [ ]:
moveClass(validationDataPath,trainDataPath,1.0)
arrX,arrY = fillArr()
fold = StratifiedKFold(n_splits=4,shuffle=True)
fold.get_n_splits(arrX,arrY)
foldIndex = 0
res
results = []
for trainIndex,labelIndex in fold.split(arrX,arrY):
    moveClass()
    foldIndex += 1
    print("Fold :",foldIndex)
    xTrain,xValue = arrX[trainIndex],arrX[labelIndex]
    yTrain,yValue = arrY[trainIndex],arrY[labelIndex]
    trainData = loadData('train')
    validData = loadData('valid')
    result = model.fit(
        trainData,
        epochs=1,
        verbose=1)
    predictions = model.predict(validData,verbose=0)
    print(predictions)
    y_predictions = np.argmax(predictions,axis=1)
    trueClasses = validData.classes
    moveClass(validationDataPath,trainDataPath,1.0)
    print("Result : ")
    acc,pre,f1 = meansure(trueClasses,y_predictions)
    results['acc'].append(acc)
    results['pre'].append(pre)
    results['f1'].append(f1)

for i in range(4):
    print("Fold : ",i)
    for k,v in results.items():
        print("Accuracy:",acc[i])
        print("Precision:",pre[i])
        print("f1-Score:",f1[i])

In [ ]:
# model.save(filepath='/home/result.h5')
